In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.0 MB/s eta 0:00:00


In [2]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.4 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
good= '/content/drive/MyDrive/Eafit - Semestre 10/Visión por computador/Talleres/Taller 11/Lemon_quality/good_quality'
bad= '/content/drive/MyDrive/Eafit - Semestre 10/Visión por computador/Talleres/Taller 11/Lemon_quality/bad_quality'

In [11]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from skopt import gp_minimize

# Directorios donde se encuentran las imágenes de calidad "good" y "bad"
good_dir = '/content/drive/MyDrive/Eafit - Semestre 10/Visión por computador/Talleres/Taller 11/Lemon_quality/good_quality'
bad_dir = '/content/drive/MyDrive/Eafit - Semestre 10/Visión por computador/Talleres/Taller 11/Lemon_quality/bad_quality'

# Lista de rutas de archivos para imágenes de buena y mala calidad
good_paths = [os.path.join(good_dir, filename) for filename in os.listdir(good_dir)]
bad_paths = [os.path.join(bad_dir, filename) for filename in os.listdir(bad_dir)]

# División de los datos en conjuntos de entrenamiento, validación y prueba
train_paths, test_paths = train_test_split(good_paths + bad_paths, test_size=0.2, random_state=42)
val_paths, test_paths = train_test_split(test_paths, test_size=0.5, random_state=42)

# Configuración del generador de imágenes
datagen = ImageDataGenerator(rescale=1./255)

# Crear un DataFrame con las rutas de las imágenes y sus respectivas etiquetas
train_df = pd.DataFrame({'filename': train_paths, 'label': [1 if 'good' in path else 0 for path in train_paths]})
val_df = pd.DataFrame({'filename': val_paths, 'label': [1 if 'good' in path else 0 for path in val_paths]})
test_df = pd.DataFrame({'filename': test_paths, 'label': [1 if 'good' in path else 0 for path in test_paths]})

# Convertir las etiquetas en cadenas en el DataFrame
train_df['label'] = train_df['label'].astype(str)
val_df['label'] = val_df['label'].astype(str)
test_df['label'] = test_df['label'].astype(str)

# Generadores de imágenes
train_generator = datagen.flow_from_dataframe(train_df, x_col='filename', y_col='label', target_size=(150, 150), batch_size=32, class_mode='binary')
val_generator = datagen.flow_from_dataframe(val_df, x_col='filename', y_col='label', target_size=(150, 150), batch_size=32, class_mode='binary')
test_generator = datagen.flow_from_dataframe(test_df, x_col='filename', y_col='label', target_size=(150, 150), batch_size=32, class_mode='binary')
# Definir la estructura del modelo
def build_model(hp):
    model = Sequential([
        Conv2D(hp.Int('conv1_filter', min_value=32, max_value=128, step=16), (3, 3), activation='relu', input_shape=(150, 150, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(hp.Int('conv2_filter', min_value=32, max_value=64, step=16), (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(hp.Int('conv3_filter', min_value=32, max_value=64, step=16), (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(hp.Int('dense_units', min_value=32, max_value=128, step=16), activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Definir el tuner para la optimización de hiperparámetros
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='keras_tuner',
    project_name='lemon_quality')

# Ejecutar la búsqueda de hiperparámetros
tuner.search(train_generator,
             validation_data=val_generator,
             epochs=5,
             verbose=1)

# Obtener el mejor modelo
best_model = tuner.get_best_models(num_models=1)[0]

# Imprimir el resumen del mejor modelo
best_model.summary()


Trial 2 Complete [00h 12m 52s]
val_accuracy: 0.9516907930374146

Best val_accuracy So Far: 0.9516907930374146
Total elapsed time: 00h 28m 52s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
96                |80                |conv1_filter
32                |48                |conv2_filter
64                |64                |conv3_filter
32                |80                |dense_units
0.001             |0.0001            |learning_rate

Epoch 1/5
52/52 [==============================] - 143s 3s/step - loss: 0.4716 - accuracy: 0.7854 - val_loss: 0.3110 - val_accuracy: 0.8889
Epoch 2/5
52/52 [==============================] - 141s 3s/step - loss: 0.3540 - accuracy: 0.8529 - val_loss: 0.2156 - val_accuracy: 0.9179
Epoch 3/5
46/52 [=========================>....] - ETA: 15s - loss: 0.1876 - accuracy: 0.9250

KeyboardInterrupt: 